In [395]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

import xgboost as xg

from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from catboost import CatBoostClassifier, Pool

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/water-pollution/water pollution (data for classification models)/Submission.csv
/kaggle/input/water-pollution/water pollution (data for classification models)/Target.csv
/kaggle/input/water-pollution/water pollution (data for classification models)/Train.csv
/kaggle/input/water-pollution/water pollution (data for classification models)/Test.csv
/kaggle/input/water-pollution/water_pollution_dataset/Submission.csv
/kaggle/input/water-pollution/water_pollution_dataset/Target.csv
/kaggle/input/water-pollution/water_pollution_dataset/Train.csv
/kaggle/input/water-pollution/water_pollution_dataset/Test.csv


In [396]:
DF_train = pd.read_csv('/kaggle/input/water-pollution/water pollution (data for classification models)/Train.csv', delimiter = ',')
DF_test = pd.read_csv('/kaggle/input/water-pollution/water pollution (data for classification models)/Test.csv', delimiter = ',')
Target = pd.read_csv('/kaggle/input/water-pollution/water pollution (data for classification models)/Target.csv', delimiter = ',')
Submission = pd.read_csv('/kaggle/input/water-pollution/water pollution (data for classification models)/Submission.csv', delimiter = ',')

### Concatinate train and test

In [397]:
df = pd.concat([DF_train, DF_test])

In [398]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3660 entries, 0 to 731
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       3660 non-null   int64  
 1   code             3660 non-null   object 
 2   year             3660 non-null   int64  
 3   period           3660 non-null   object 
 4   id               3660 non-null   object 
 5   Country          3660 non-null   object 
 6   tourists         3660 non-null   float64
 7   venue            3660 non-null   float64
 8   rate             3660 non-null   float64
 9   food             3660 non-null   float64
 10  glass            3660 non-null   float64
 11  metal            3660 non-null   float64
 12  other            3660 non-null   float64
 13  paper            3660 non-null   float64
 14  plastic          3660 non-null   float64
 15  leather          3660 non-null   float64
 16  green_waste      3660 non-null   float64
 17  waste_recycling

## View categorical and numerical columns


In [399]:
cat_columns = []
num_columns = []

for column_name in df.columns:
    if (df[column_name].dtypes == object):
        cat_columns +=[column_name]
    else:
        num_columns +=[column_name]

print('categorical columns:\t ',cat_columns, '\n len = ',len(cat_columns))

print('numerical columns:\t ',  num_columns, '\n len = ',len(num_columns))

categorical columns:	  ['code', 'period', 'id', 'Country'] 
 len =  4
numerical columns:	  ['Unnamed: 0', 'year', 'tourists', 'venue', 'rate', 'food', 'glass', 'metal', 'other', 'paper', 'plastic', 'leather', 'green_waste', 'waste_recycling'] 
 len =  14


### Move 'year' to num columns


In [400]:
cat_columns = ['code', 'period', 'id', 'Country', 'year']
num_columns = ['tourists', 'venue', 'rate', 'food', 'glass', 'metal', 'other', 'paper', 'plastic', 'leather', 'green_waste', 'waste_recycling'] 

# OHE

In [401]:
df_ohe = pd.get_dummies(df[cat_columns])

In [402]:
df_all = pd.concat([df_ohe, df[num_columns]], axis=1)

# Train test split

In [403]:
# Creating StandardScaler Object
scaler = StandardScaler()
df_scale = pd.DataFrame(scaler.fit_transform(df_all[num_columns]))
df_all = pd.concat([df_ohe.reset_index(drop=True), df_scale.reset_index(drop=True)], axis=1)

In [404]:
len_test = len(DF_test)
train = df_all.iloc[:-len_test]
test = df_all.iloc[-len_test:]
print(DF_train.shape[0], train.shape[0], DF_test.shape[0], test.shape[0])

2928 2928 732 732


In [405]:
#X, y = pd.DataFrame(train).values, Target['polution_clf'].values
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#X_train.shape[0], X_test.shape[0], y_train.shape[0], y_test.shape[0]

In [406]:
print(DF_train.shape[0], train.shape[0], DF_test.shape[0], test.shape[0])

2928 2928 732 732


## Preprocessing 

In [407]:
from sklearn.preprocessing import PowerTransformer

from sklearn.impute import SimpleImputer

from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, OrdinalEncoder

In [408]:
X_train = DF_train.copy()
X_test = DF_test.copy()
y_train = Target['polution_clf']

In [409]:
X_train_, X_val, y_train_, y_val = train_test_split(X_train,y_train,test_size = 0.1, random_state = 34)

In [410]:
X_train_.shape, X_val.shape, y_train_.shape, y_val.shape, X_train.shape, X_test.shape, y_train.shape

((2635, 18), (293, 18), (2635,), (293,), (2928, 18), (732, 18), (2928,))

### Model scoring function

In [411]:
def calculate_f1_score(model_pipe, X, y):
    """F1 calculation. 
    
    Parameters:
    ===========
    model_pipe: model or pipeline
    X: features
    y: real values
    """
    y_model = model_pipe.predict(X)
    return f1_score(y, y_model)

# Data preparation pipeline

In [412]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

Numerical data preprocessing (insert missing values and normalization) 

In [413]:
numerical_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
   ('scaler', MinMaxScaler())
])
numerical_pipe

Pipeline(steps=[('imputer', SimpleImputer(strategy='median')),
                ('scaler', MinMaxScaler())])

Categorical data preprocessing (insert missing values + OHE)

In [414]:
categorical_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent', )),
    ('encoder', OneHotEncoder(drop='if_binary', handle_unknown='ignore', sparse=False))
])
categorical_pipe

Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),
                ('encoder',
                 OneHotEncoder(drop='if_binary', handle_unknown='ignore',
                               sparse=False))])

### Concatinate preprocessing pipilines

In [415]:
preprocessors = ColumnTransformer(transformers=[
    ('num', numerical_pipe, num_columns[1:]),
    ('cat', categorical_pipe, cat_columns)
])
preprocessors

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', MinMaxScaler())]),
                                 ['venue', 'rate', 'food', 'glass', 'metal',
                                  'other', 'paper', 'plastic', 'leather',
                                  'green_waste', 'waste_recycling']),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('encoder',
                                                  OneHotEncoder(drop='if_binary',
                                                                handle_unknown='ignore',
                                                                sparse=False))]),
       

### Concatinate preprocessing pipeline and model

In [416]:
pipe = Pipeline([
    ('preprocessors', preprocessors),
    ('model', LogisticRegression(C=2, random_state=42, max_iter = 100000))
])

In [417]:
pipe.fit(X_train_, y_train_)

Pipeline(steps=[('preprocessors',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   MinMaxScaler())]),
                                                  ['venue', 'rate', 'food',
                                                   'glass', 'metal', 'other',
                                                   'paper', 'plastic',
                                                   'leather', 'green_waste',
                                                   'waste_recycling']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                         

# LogisticRegression

In [418]:
pred = pipe.predict(X_test)
Submission.polution_clf = pred

/opt/conda/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:174: UserWarning: Found unknown categories in columns [1, 2, 4] during transform. These unknown categories will be encoded as all zeros
  UserWarning,


In [419]:
from sklearn.metrics import accuracy_score as acc


In [420]:
Target.polution_clf.shape

(2928,)

In [421]:
print(f"Train R2: {calculate_f1_score(pipe, X_train_, y_train_):.4f}")

print(f"Val R2: {calculate_f1_score(pipe, X_val, y_val):.4f}")
print('F1:', f1_score(Target.polution_clf, pred))


/opt/conda/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:174: UserWarning: Found unknown categories in columns [1, 2, 4] during transform. These unknown categories will be encoded as all zeros
  UserWarning,


Train R2: 0.9283
Val R2: 0.8328


ValueError: Found input variables with inconsistent numbers of samples: [2928, 732]

# XGBoost

# CatBoost

## Sumbission results

In [422]:
! rm '/kaggle/working/water-pollution/water pollution (data for classification models)/Submission.csv'
! mkdir '/kaggle/working/water-pollution/water pollution (data for classification models)/'

mkdir: cannot create directory ‘/kaggle/working/water-pollution/water pollution (data for classification models)/’: File exists


In [423]:
Submission.to_csv('/kaggle/working/water-pollution/water pollution (data for classification models)/Submission.csv',index=False)

In [424]:
! mkdir ~/.kaggle
! touch ~/.kaggle/kaggle.json
! echo '{"username":"hydramst","key": "4f8064cecf738a0c9c4a362dc0e1ed92" }' > ~/.kaggle/kaggle.json
! chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [425]:
cat ~/.kaggle/kaggle.json

{"username":"hydramst","key": "4f8064cecf738a0c9c4a362dc0e1ed92" }


In [426]:
! kaggle competitions submit -c sf-matml-2022-classification -f '/kaggle/working/water-pollution/water pollution (data for classification models)/Submission.csv' -m "Message"

100%|██████████████████████████████████████| 5.30k/5.30k [00:00<00:00, 26.5kB/s]
Successfully submitted to SF_MatML_2022_Classification